In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import tkinter as tk
from tkinter import messagebox, scrolledtext
import matplotlib.pyplot as plt

# 1. Generate Synthetic Data
num_samples = 1000

# Define job options and their corresponding weight factors
job_weight_factor = {
    "Office worker": 0.95,
    "Laborer": 1.05,
    "Athlete": 1.15,
    "Teacher": 0.98,
    "Doctor": 1.00,
    "Engineer": 0.99,
    "Driver": 1.04,
    "Chef": 1.02,
    "Nurse": 1.01,
    "Construction worker": 1.10,
    "Farmer": 1.12,
    "Artist": 0.97,
    "Student": 0.92,
    "Lawyer": 0.96,
    "Scientist": 0.98,
    "Police officer": 1.08,
    "Firefighter": 1.10,
    "Pilot": 1.03,
    "Musician": 0.95,
    "Retail worker": 0.99
}

# Generate random heights and jobs
heights = np.random.uniform(1.5, 1.8, num_samples)
jobs = np.random.choice(list(job_weight_factor.keys()), num_samples)
base_weights = np.array([50, 55, 75])
base_heights = np.array([1.5, 1.6, 1.7])

def calculate_weight(height):
    return np.interp(height, base_heights, base_weights)

weights = [calculate_weight(h) * job_weight_factor[j] for h, j in zip(heights, jobs)]

# Create a DataFrame
df = pd.DataFrame({
    'Height': heights,
    'Job': jobs,
    'Weight': weights
})
df['Job_Factor'] = df['Job'].apply(lambda x: job_weight_factor[x])
X = df[['Height', 'Job_Factor']].values
y = df['Weight'].values

# Reshape for CNN and RNN
X_cnn_rnn = X.reshape(-1, 2, 1)
y_reshaped = y.reshape(-1, 1)

# 2. CNN and RNN Models (non-Keras)
class CNNModel(tf.Module):
    def __init__(self):
        self.conv1 = tf.Variable(tf.random.normal([2, 1, 32]))
        self.dense1 = tf.Variable(tf.random.normal([32, 64]))
        self.output_layer = tf.Variable(tf.random.normal([64, 1]))

    def __call__(self, x):
        conv_output = tf.nn.conv1d(x, self.conv1, stride=1, padding='VALID')
        flat_output = tf.reshape(conv_output, [-1, 32])
        dense_output = tf.nn.relu(tf.matmul(flat_output, self.dense1))
        return tf.matmul(dense_output, self.output_layer)

class RNNModel(tf.Module):
    def __init__(self):
        self.W_xh = tf.Variable(tf.random.normal([1, 64]))
        self.W_hh = tf.Variable(tf.random.normal([64, 64]))
        self.b_h = tf.Variable(tf.zeros([64]))
        self.output_layer = tf.Variable(tf.random.normal([64, 1]))
        self.b_out = tf.Variable(tf.zeros([1]))

    def __call__(self, x):
        batch_size = tf.shape(x)[0]
        hidden_state = tf.zeros([batch_size, 64])
        for t in range(x.shape[1]):
            input_t = x[:, t, :]
            hidden_state = tf.tanh(tf.matmul(input_t, self.W_xh) + tf.matmul(hidden_state, self.W_hh) + self.b_h)
        return tf.matmul(hidden_state, self.output_layer) + self.b_out

# Keras-based CNN model
def create_keras_cnn():
    model = models.Sequential([
        layers.Conv1D(32, kernel_size=2, activation='relu', input_shape=(2, 1)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Keras-based RNN model
def create_keras_rnn():
    model = models.Sequential([
        layers.SimpleRNN(64, activation='tanh', input_shape=(2, 1)),
        layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Instantiate models
cnn_model = CNNModel()
rnn_model = RNNModel()
keras_cnn_model = create_keras_cnn()
keras_rnn_model = create_keras_rnn()

# Optimizers for non-Keras models
cnn_optimizer = tf.optimizers.Adam(0.001)
rnn_optimizer = tf.optimizers.Adam(0.001)
loss_fn = tf.losses.MeanSquaredError()

# Training loops for CNN and RNN (non-Keras)
def train_cnn(X_train, y_train, epochs=100):
    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            predictions = cnn_model(tf.convert_to_tensor(X_train, dtype=tf.float32))
            loss = loss_fn(y_train, predictions)
        gradients = tape.gradient(loss, cnn_model.trainable_variables)
        cnn_optimizer.apply_gradients(zip(gradients, cnn_model.trainable_variables))

def train_rnn(X_train, y_train, epochs=100):
    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            predictions = rnn_model(tf.convert_to_tensor(X_train, dtype=tf.float32))
            loss = loss_fn(y_train, predictions)
        gradients = tape.gradient(loss, rnn_model.trainable_variables)
        rnn_optimizer.apply_gradients(zip(gradients, rnn_model.trainable_variables))

# Train Keras models
keras_cnn_model.fit(X_cnn_rnn, y_reshaped, epochs=100, verbose=0)
keras_rnn_model.fit(X_cnn_rnn, y_reshaped, epochs=100, verbose=0)

# Train non-Keras models
train_cnn(X_cnn_rnn, y_reshaped)
train_rnn(X_cnn_rnn, y_reshaped)

# 3. Model Comparison and GUI Update
def predict_weight_and_bmi_category():
    try:
        # Get user inputs
        height = float(entry_height.get())
        job = job_var.get()

        # Calculate job factor
        job_factor = job_weight_factor[job]

        # Prepare input for the model
        input_data = np.array([[height, job_factor]]).reshape(-1, 2, 1)

        # Select the model based on user selection
        selected_model = model_var.get()
        if selected_model == "CNN (non-Keras)":
            prediction = cnn_model(tf.convert_to_tensor(input_data, dtype=tf.float32)).numpy().flatten()[0]
        elif selected_model == "RNN (non-Keras)":
            prediction = rnn_model(tf.convert_to_tensor(input_data, dtype=tf.float32)).numpy().flatten()[0]
        elif selected_model == "CNN (Keras)":
            prediction = keras_cnn_model.predict(input_data).flatten()[0]
        elif selected_model == "RNN (Keras)":
            prediction = keras_rnn_model.predict(input_data).flatten()[0]
        else:
            prediction = "Model not found"

        # Calculate BMI
        bmi = prediction / (height ** 2)
        if bmi < 18.5:
            bmi_category = "Underweight"
            advice = "You are underweight. It's important to eat a balanced diet with more calories to gain weight. Consult a nutritionist for a proper plan."
        elif 18.5 <= bmi <= 25:
            bmi_category = "Normal weight"
            advice = "You have a normal weight. Maintain a balanced diet and regular exercise to keep your healthy weight."
        else:
            bmi_category = "Overweight"
            advice = "You are overweight. It's advisable to adopt a healthier diet with fewer calories and increase physical activity. Consider consulting a healthcare provider."
            
        # Display the result in chatbot
        result_text = f"Predicted Weight: {prediction:.2f} kg\nBMI: {bmi:.2f}\nBMI Category: {bmi_category}\n\nHealth Advice: {advice}"
        chatbot_display.insert(tk.END, "Chatbot: " + result_text + "\n\n")
        chatbot_display.see(tk.END)
        
    except Exception as e:
        messagebox.showerror("Error", f"Invalid input: {e}")


def compare_models():
    # Predictions
    cnn_predictions = cnn_model(tf.convert_to_tensor(X_cnn_rnn, dtype=tf.float32)).numpy().flatten()
    rnn_predictions = rnn_model(tf.convert_to_tensor(X_cnn_rnn, dtype=tf.float32)).numpy().flatten()
    keras_cnn_predictions = keras_cnn_model.predict(X_cnn_rnn).flatten()
    keras_rnn_predictions = keras_rnn_model.predict(X_cnn_rnn).flatten()

    # Metrics
    metrics = {}
    for name, predictions in zip(['CNN (non-Keras)', 'RNN (non-Keras)', 'CNN (Keras)', 'RNN (Keras)'],
                                 [cnn_predictions, rnn_predictions, keras_cnn_predictions, keras_rnn_predictions]):
        mse = mean_squared_error(y, predictions)
        mae = mean_absolute_error(y, predictions)
        rmse = np.sqrt(mse)
        mape = mean_absolute_percentage_error(y, predictions)
        metrics[name] = {'MSE' : mse, 'MAE': mae, 'RMSE': rmse, 'MAPE': mape}

    # Display Metrics
    fig, ax = plt.subplots()
    x = np.arange(len(metrics['CNN (non-Keras)']))
    width = 0.2

    for i, (model, vals) in enumerate(metrics.items()):
        ax.bar(x + width * i, list(vals.values()), width, label=model)

    ax.set_xticks(x)
    ax.set_xticklabels(['MSE', 'MAE', 'RMSE', 'MAPE'])
    ax.legend()
    plt.show()

# 4. GUI Setup (unchanged)
# Same as your previous code for chatbot interaction and prediction

root = tk.Tk()
root.title("Health Prediction Chatbot")

# Chatbot interface
chatbot_display = scrolledtext.ScrolledText(root, width=60, height=20)
chatbot_display.grid(row=0, column=0, columnspan=4, padx=10, pady=10)

# User input fields
tk.Label(root, text="Enter height (m):").grid(row=1, column=0)
entry_height = tk.Entry(root)
entry_height.grid(row=1, column=1)

tk.Label(root, text="Select job:").grid(row=1, column=2)
job_var = tk.StringVar()
job_var.set("Office worker")
job_menu = tk.OptionMenu(root, job_var, *job_weight_factor.keys())
job_menu.grid(row=1, column=3)

tk.Label(root, text="Select model:").grid(row=2, column=0)
model_var = tk.StringVar()
model_var.set("CNN (non-Keras)")
model_menu = tk.OptionMenu(root, model_var, "CNN (non-Keras)", "RNN (non-Keras)", "CNN (Keras)", "RNN (Keras)")
model_menu.grid(row=2, column=1)

# Prediction and compare buttons
predict_button = tk.Button(root, text="Predict Weight & BMI", command=predict_weight_and_bmi_category)
predict_button.grid(row=2, column=2, pady=10)

compare_button = tk.Button(root, text="Compare Models", command=compare_models)
compare_button.grid(row=2, column=3, pady=10)

root.mainloop()




C:\Users\84395\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\84395\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import tkinter as tk
from tkinter import messagebox, scrolledtext
import matplotlib.pyplot as plt

# 1. Generate Synthetic Data
num_samples = 1000

# Define job options and their corresponding weight factors
job_weight_factor = {
    "Office worker": 0.95,
    "Laborer": 1.05,
    "Athlete": 1.15,
    "Teacher": 0.98,
    "Doctor": 1.00,
    "Engineer": 0.99,
    "Driver": 1.04,
    "Chef": 1.02,
    "Nurse": 1.01,
    "Construction worker": 1.10,
    "Farmer": 1.12,
    "Artist": 0.97,
    "Student": 0.92,
    "Lawyer": 0.96,
    "Scientist": 0.98,
    "Police officer": 1.08,
    "Firefighter": 1.10,
    "Pilot": 1.03,
    "Musician": 0.95,
    "Retail worker": 0.99
}

# Generate random heights and jobs
heights = np.random.uniform(1.5, 1.8, num_samples)
jobs = np.random.choice(list(job_weight_factor.keys()), num_samples)
base_weights = np.array([50, 55, 75])
base_heights = np.array([1.5, 1.6, 1.7])

def calculate_weight(height):
    return np.interp(height, base_heights, base_weights)

weights = [calculate_weight(h) * job_weight_factor[j] for h, j in zip(heights, jobs)]

# Create a DataFrame
df = pd.DataFrame({
    'Height': heights,
    'Job': jobs,
    'Weight': weights
})
df['Job_Factor'] = df['Job'].apply(lambda x: job_weight_factor[x])
X = df[['Height', 'Job_Factor']].values
y = df['Weight'].values

# Reshape for CNN and RNN
X_cnn_rnn = X.reshape(-1, 2, 1)
y_reshaped = y.reshape(-1, 1)

# 2. CNN and RNN Models (non-Keras)
class CNNModel(tf.Module):
    def __init__(self):
        self.conv1 = tf.Variable(tf.random.normal([2, 1, 32]))
        self.dense1 = tf.Variable(tf.random.normal([32, 64]))
        self.output_layer = tf.Variable(tf.random.normal([64, 1]))

    def __call__(self, x):
        conv_output = tf.nn.conv1d(x, self.conv1, stride=1, padding='VALID')
        flat_output = tf.reshape(conv_output, [-1, 32])
        dense_output = tf.nn.relu(tf.matmul(flat_output, self.dense1))
        return tf.matmul(dense_output, self.output_layer)

class RNNModel(tf.Module):
    def __init__(self):
        self.W_xh = tf.Variable(tf.random.normal([1, 64]))
        self.W_hh = tf.Variable(tf.random.normal([64, 64]))
        self.b_h = tf.Variable(tf.zeros([64]))
        self.output_layer = tf.Variable(tf.random.normal([64, 1]))
        self.b_out = tf.Variable(tf.zeros([1]))

    def __call__(self, x):
        batch_size = tf.shape(x)[0]
        hidden_state = tf.zeros([batch_size, 64])
        for t in range(x.shape[1]):
            input_t = x[:, t, :]
            hidden_state = tf.tanh(tf.matmul(input_t, self.W_xh) + tf.matmul(hidden_state, self.W_hh) + self.b_h)
        return tf.matmul(hidden_state, self.output_layer) + self.b_out

# Keras-based CNN model
def create_keras_cnn():
    model = models.Sequential([
        layers.Conv1D(32, kernel_size=2, activation='relu', input_shape=(2, 1)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Keras-based RNN model
def create_keras_rnn():
    model = models.Sequential([
        layers.SimpleRNN(64, activation='tanh', input_shape=(2, 1)),
        layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Instantiate models
cnn_model = CNNModel()
rnn_model = RNNModel()
keras_cnn_model = create_keras_cnn()
keras_rnn_model = create_keras_rnn()

# Optimizers for non-Keras models
cnn_optimizer = tf.optimizers.Adam(0.001)
rnn_optimizer = tf.optimizers.Adam(0.001)
loss_fn = tf.losses.MeanSquaredError()

# Training loops for CNN and RNN (non-Keras)
def train_cnn(X_train, y_train, epochs=100):
    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            predictions = cnn_model(tf.convert_to_tensor(X_train, dtype=tf.float32))
            loss = loss_fn(y_train, predictions)
        gradients = tape.gradient(loss, cnn_model.trainable_variables)
        cnn_optimizer.apply_gradients(zip(gradients, cnn_model.trainable_variables))

def train_rnn(X_train, y_train, epochs=100):
    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            predictions = rnn_model(tf.convert_to_tensor(X_train, dtype=tf.float32))
            loss = loss_fn(y_train, predictions)
        gradients = tape.gradient(loss, rnn_model.trainable_variables)
        rnn_optimizer.apply_gradients(zip(gradients, rnn_model.trainable_variables))

# Train Keras models
keras_cnn_model.fit(X_cnn_rnn, y_reshaped, epochs=100, verbose=0)
keras_rnn_model.fit(X_cnn_rnn, y_reshaped, epochs=100, verbose=0)

# Train non-Keras models
train_cnn(X_cnn_rnn, y_reshaped)
train_rnn(X_cnn_rnn, y_reshaped)

# 3. Model Comparison and GUI Update
def predict_weight_and_bmi_category():
    try:
        # Get user inputs
        height = float(entry_height.get())
        job = job_var.get()

        # Calculate job factor
        job_factor = job_weight_factor[job]

        # Prepare input for the model
        input_data = np.array([[height, job_factor]]).reshape(-1, 2, 1)

        # Select the model based on user selection
        selected_model = model_var.get()
        if selected_model == "CNN (non-Keras)":
            prediction = cnn_model(tf.convert_to_tensor(input_data, dtype=tf.float32)).numpy().flatten()[0]
        elif selected_model == "RNN (non-Keras)":
            prediction = rnn_model(tf.convert_to_tensor(input_data, dtype=tf.float32)).numpy().flatten()[0]
        elif selected_model == "CNN (Keras)":
            prediction = keras_cnn_model.predict(input_data).flatten()[0]
        elif selected_model == "RNN (Keras)":
            prediction = keras_rnn_model.predict(input_data).flatten()[0]
        else:
            prediction = "Model not found"

        # Calculate BMI
        bmi = prediction / (height ** 2)
        if bmi < 18.5:
            bmi_category = "Underweight"
            advice = "You are underweight. It's important to eat a balanced diet with more calories to gain weight. Consult a nutritionist for a proper plan."
        elif 18.5 <= bmi <= 25:
            bmi_category = "Normal weight"
            advice = "You have a normal weight. Maintain a balanced diet and regular exercise to keep your healthy weight."
        else:
            bmi_category = "Overweight"
            advice = "You are overweight. It's advisable to adopt a healthier diet with fewer calories and increase physical activity. Consider consulting a healthcare provider."
            
        # Display the result in chatbot
        result_text = f"Predicted Weight: {prediction:.2f} kg\nBMI: {bmi:.2f}\nBMI Category: {bmi_category}\n\nHealth Advice: {advice}"
        chatbot_display.insert(tk.END, "Chatbot: " + result_text + "\n\n")
        chatbot_display.see(tk.END)
        
    except Exception as e:
        messagebox.showerror("Error", f"Invalid input: {e}")


def compare_models():
    # Predictions
    cnn_predictions = cnn_model(tf.convert_to_tensor(X_cnn_rnn, dtype=tf.float32)).numpy().flatten()
    rnn_predictions = rnn_model(tf.convert_to_tensor(X_cnn_rnn, dtype=tf.float32)).numpy().flatten()
    keras_cnn_predictions = keras_cnn_model.predict(X_cnn_rnn).flatten()
    keras_rnn_predictions = keras_rnn_model.predict(X_cnn_rnn).flatten()

    # Metrics
    metrics = {}
    for name, predictions in zip(['CNN (non-Keras)', 'RNN (non-Keras)', 'CNN (Keras)', 'RNN (Keras)'],
                                 [cnn_predictions, rnn_predictions, keras_cnn_predictions, keras_rnn_predictions]):
        mae = mean_absolute_error(y, predictions)
        rmse = np.sqrt(mean_squared_error(y, predictions))
        mape = mean_absolute_percentage_error(y, predictions)
        rme = np.mean(y - predictions)  # Tính phần dư trung bình (Residual Mean Error)
        metrics[name] = {'MAE': mae, 'RMSE': rmse, 'MAPE': mape, 'RME': rme}

    # Display Metrics
    fig, ax = plt.subplots()
    x = np.arange(len(metrics['CNN (non-Keras)']))
    width = 0.2

    # Biểu đồ các chỉ số so sánh
    for i, (model, vals) in enumerate(metrics.items()):
        ax.bar(x + width * i, list(vals.values()), width, label=model)

    ax.set_xticks(x)
    ax.set_xticklabels(['MAE', 'RMSE', 'MAPE', 'RME'])
    ax.legend()
    plt.show()

# 4. GUI Setup (unchanged)
# Same as your previous code for chatbot interaction and prediction

root = tk.Tk()
root.title("Health Prediction Chatbot")

# Chatbot interface
chatbot_display = scrolledtext.ScrolledText(root, width=60, height=20)
chatbot_display.grid(row=0, column=0, columnspan=4, padx=10, pady=10)

# User input fields
tk.Label(root, text="Enter height (m):").grid(row=1, column=0)
entry_height = tk.Entry(root)
entry_height.grid(row=1, column=1)

tk.Label(root, text="Select job:").grid(row=1, column=2)
job_var = tk.StringVar()
job_var.set("Office worker")
job_menu = tk.OptionMenu(root, job_var, *job_weight_factor.keys())
job_menu.grid(row=1, column=3)

tk.Label(root, text="Select model:").grid(row=2, column=0)
model_var = tk.StringVar()
model_var.set("CNN (non-Keras)")
model_menu = tk.OptionMenu(root, model_var, "CNN (non-Keras)", "RNN (non-Keras)", "CNN (Keras)", "RNN (Keras)")
model_menu.grid(row=2, column=1)

# Prediction and compare buttons
predict_button = tk.Button(root, text="Predict Weight & BMI", command=predict_weight_and_bmi_category)
predict_button.grid(row=2, column=2, pady=10)

compare_button = tk.Button(root, text="Compare Models", command=compare_models)
compare_button.grid(row=2, column=3, pady=10)

root.mainloop()




C:\Users\84395\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\84395\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
